## 1. Installing the Mario environment

In [ ]:
!pip install gym-super-mario-bros

In [ ]:
from nes_py.wrappers import JoypadSpace
JoypadSpace.reset = lambda self, **kwargs: self.env.reset(**kwargs)
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [ ]:
env = gym_super_mario_bros.make('SuperMarioBros-v0',apply_api_compatibility=True,render_mode='human')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

done = True
for step in range(5000):
    if done:
        state = env.reset()
    state, reward, done, truncate, info = env.step(env.action_space.sample())
    env.render()

env.close()

## 2. Preprocessing The Environment

In [ ]:
#Installing PyTorch
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
#Installing Stable-Baselines3
!pip install stable-baselines3[extra]

In [ ]:
# Importing GrayScaling Wrapper
from gym.wrappers import GrayScaleObservation
# Importing Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv

In [ ]:
# Creating the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode = 'human')
# Simplifying the controls 
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# Grayscaling
env = GrayScaleObservation(env, keep_dim=True)
# Wraping inside the Dummy Environment
env = DummyVecEnv([lambda: env])
# Stacking the frames
env = VecFrameStack(env, 4, channels_order='last')

## 3. Training the RL Model

In [ ]:
# Importing os for file management
import os
# Importing PPO as the algo
from stable_baselines3 import PPO
# Importing BaseCallback to save models
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'model_{}'.format(self.n_calls))
            self.model.save(model_path)
        return True

In [ ]:
# Path where the model is saved
checkpoint_dir = './models/'
# Path where logs are saved
log_dir = './logs/'

In [ ]:
# Setting up callback
callback = TrainAndLoggingCallback(check_freq = 10000, save_path = checkpoint_dir)

In [ ]:
# Creating the model
model = PPO('CnnPolicy', env, verbose = 1, tensorboard_log = log_dir, learning_rate = 0.00001, n_steps = 512)

In [ ]:
# Training the model
model.learn(total_timesteps = 1000000, callback = callback)

## 4. Evaluating the Model (1,000,000 Timesteps)

In [ ]:
# Selecting the best model
model = PPO.load('./models/model_850000')

In [ ]:
# Creating the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode = 'human')
# Simplifying the controls 
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# Grayscaling
env = GrayScaleObservation(env, keep_dim=True)
# Wraping inside the Dummy Environment
env = DummyVecEnv([lambda: env])
# Stacking the frames
env = VecFrameStack(env, 4, channels_order='last')
# Starting the environment
state = env.reset()

In [ ]:
while True:
    action, _state = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()

In [ ]:
env.close()

## 5. Fine-Tuning the Best Model (1,000,000 to 2,000,000 Timesteps)

In [ ]:
# Creating the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode = 'human')
# Simplifying the controls 
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# Grayscaling
env = GrayScaleObservation(env, keep_dim=True)
# Wraping inside the Dummy Environment
env = DummyVecEnv([lambda: env])
# Stacking the frames
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
# Changing the dirs
checkpoint_dir = './models/850000/'
log_dir = './logs/850000/'

In [ ]:
# Setting up callback
callback = TrainAndLoggingCallback(check_freq = 10000, save_path = checkpoint_dir)

In [ ]:
# Setting the hyperparameters of the model to train further
model.set_env(env)
model.tensorboard_log = log_dir
model.learning_rate = 0.00001

In [ ]:
# Training the previously selected best model
model.learn(total_timesteps = 1000000, callback = callback)

## 6. Evaluating the Fine-Tuned Model (2,000,000 Timesteps)

In [ ]:
# Selecting the best model
model = PPO.load('./models/850000/model_930000')

In [ ]:
# Creating the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode = 'human')
# Simplifying the controls 
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# Grayscaling
env = GrayScaleObservation(env, keep_dim=True)
# Wraping inside the Dummy Environment
env = DummyVecEnv([lambda: env])
# Stacking the frames
env = VecFrameStack(env, 4, channels_order='last')
# Starting the environment
state = env.reset()

In [ ]:
while True:
    action, _state = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()

In [ ]:
env.close()

## 7. Advanced Fine-Tuning (2,000,000 to 3,000,000 Timesteps)

In [ ]:
# Creating the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode = 'human')
# Simplifying the controls 
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# Grayscaling
env = GrayScaleObservation(env, keep_dim=True)
# Wraping inside the Dummy Environment
env = DummyVecEnv([lambda: env])
# Stacking the frames
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
# Changing the dirs
checkpoint_dir = './models/850000/930000/'
log_dir = './logs/850000/930000/'

In [ ]:
# Setting up callback
callback = TrainAndLoggingCallback(check_freq = 10000, save_path = checkpoint_dir)

In [ ]:
# Setting the hyperparameters of the model to train further
model.set_env(env)
model.tensorboard_log = log_dir
model.learning_rate = 0.00001

In [ ]:
# Training the previously selected best model
model.learn(total_timesteps = 1000000, callback = callback)

## 8. Evaluating the Advanced Fine-Tuned Model (3,000,000 Timesteps)

In [ ]:
# Selecting the best model
model = PPO.load('./models/850000/930000/model_930000')

In [ ]:
# Creating the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode = 'human')
# Simplifying the controls 
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# Grayscaling
env = GrayScaleObservation(env, keep_dim=True)
# Wraping inside the Dummy Environment
env = DummyVecEnv([lambda: env])
# Stacking the frames
env = VecFrameStack(env, 4, channels_order='last')
# Starting the environment
state = env.reset()

In [ ]:
while True:
    action, _state = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()

In [ ]:
env.close()

## 9. Final Fine-Tuning (3,000,000 to 4,000,000 Timesteps)

In [ ]:
# Creating the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode = 'human')
# Simplifying the controls 
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# Grayscaling
env = GrayScaleObservation(env, keep_dim=True)
# Wraping inside the Dummy Environment
env = DummyVecEnv([lambda: env])
# Stacking the frames
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
checkpoint_dir = './models/850000/930000/930000/'
log_dir = './logs/850000/930000/930000/'

In [ ]:
# Setting up callback
callback = TrainAndLoggingCallback(check_freq = 10000, save_path = checkpoint_dir)

In [ ]:
# Setting the hyperparameters of the model to train further
model.set_env(env)
model.tensorboard_log = log_dir
model.learning_rate = 0.00001

In [ ]:
# Training the previously selected best model
model.learn(total_timesteps = 1000000, callback = callback)

## 10. Evaluating the Final Fine-Tuned Model (4,000,000 Timesteps)

In [ ]:
# Selecting the best model
model = PPO.load('./models/850000/930000/930000/model_750000/')

In [ ]:
# Creating the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode = 'human')
# Simplifying the controls 
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# Grayscaling
env = GrayScaleObservation(env, keep_dim=True)
# Wraping inside the Dummy Environment
env = DummyVecEnv([lambda: env])
# Stacking the frames
env = VecFrameStack(env, 4, channels_order='last')
# Starting the environment
state = env.reset()

In [ ]:
while True:
    action, _state = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()

In [ ]:
env.close()